In [1]:
# !pip install import-ipynb

In [1]:
import pandas as pd
import numpy as np
import pyodbc
from scipy.stats import shapiro
from scipy.stats import kstest
from datetime import datetime
from configparser import ConfigParser

# import import_ipynb
# import Entity_Risk

In [2]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [3]:
# clustered_data = pd.read_csv("/data/Synthetic_data_Clustered.csv")
data_ind_U = pd.read_csv(r"/data/peer_updated_Incremental.csv")
#Uncomment to run on org
# data_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\Synthetic_data_Clustered_organization_U.csv")

# data_ind = pd.read_csv("/data/Synthetic_data_Clustered_individual.csv")
# data_org = pd.read_csv("/data/Synthetic_data_Clustered_organization.csv")

In [4]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [5]:
# #Create dataframe using pyodbc
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# # cursor1 = conn.cursor()
# # cursor2 = conn.cursor()


In [6]:

risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES]", conn)
static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1)]['RULE_NO'].to_list()

auto_high_rules =  risk_lkp[(risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['RISK_TYPE'] == 'Auto High Rules ')]['RULE_NO'].to_list()

non_stat_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 0) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Organization')]['RULE_NO'].to_list()

non_stat_rules = tuple(non_stat_rules)

stat_rules = tuple(static_rules)

In [13]:
non_stat_rules

()

In [8]:
et_risk_df_per = pd.read_sql_query("SELECT * FROM FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG WHERE ORGANIZATION_MASTER_ID IN (SELECT ORGANIZATION_Master_Id FROM FINCORE_DB.Fincore_Analytics.risk_organization_incremental )", conn)

In [9]:
et_risk_df_per

ORGANIZATION_MASTER_ID PEER_GROUP_ID         PAN GSTIN UNIQUE_COMPANY_ID  \
0                      29      Others_0  ACCGS1203J  None              None   
1                      30      Others_0  CFBNM9876K  None              None   
2                      31      Others_0  PGJPK8013L  None              None   
3                      32      Others_0  ASDPS1234M  None              None   
4                      33      Others_0  ACCGS1203J  None              None   
5                      34      Others_0  CFBNM9876K  None              None   
6                      35      Others_0  PGJPK8013L  None              None   
7                      36      Others_0  ASDPS1234M  None              None   
8                      37      Others_0  ACCGS1203J  None              None   

    IEC      FULL_NAME DATE_OF_INCORPORATION CBDT_INCOME_RANGE  \
0  None           None                  None              None   
1  None           None                  None              None   
2  None  Vidhyut Varma                  None              None   
3  None           None                  None              None   
4  None           None                  None              None   
5  None           None                  None              None   
6  None  Vidhyut Varma                  None              None   
7  None           None                  None              None   
8  None           None                  None              None   

   TRAN_AMNT_CREDIT TRAN_AMNT_DEBIT  PEER_GROUP_AMNT PINCODE LATITUDE  \
0       10000.00000            None      10000.00000    None     None   
1       10000.00000            None      10000.00000    None     None   
2       10000.00000            None      10000.00000    None     None   
3       10000.00000            None      10000.00000    None     None   
4       10000.00000            None      10000.00000    None     None   
5       10000.00000            None      10000.00000    None     None   
6       10000.00000            None      10000.00000    None     None   
7       10000.00000            None      10000.00000    None     None   
8       10000.00000            None      10000.00000    None     None   

  LONGITUDE OCCUPATION_BUSINESS ACCOUNT_COUNT ADDRESS_COUNT  MOBILE_COUNT  \
0      None                None          None          None             1   
1      None                None          None          None             1   
2      None                None          None          None             1   
3      None                None          None          None             1   
4      None                None          None          None             1   
5      None                None          None          None             1   
6      None                None          None          None             1   
7      None                None          None          None             1   
8      None                None          None          None             1   

  TELEPHONE_COUNT EMAIL_COUNT WATCHLIST_COUNT LEA_REQUESTS_COUNT  \
0            None        None            None               None   
1            None        None            None               None   
2            None        None            None               None   
3            None        None            None               None   
4            None        None            None               None   
5            None        None            None               None   
6            None        None            None               None   
7            None        None            None               None   
8            None        None            None               None   

  LNK_REPORT_COUNT STR_REPORT_COUNT STR_AMOUNT_DEBIT STR_AMOUNT_CREDIT  \
0             None             None             None              None   
1             None             None             None              None   
2             None             None             None              None   
3             None             None             None              None  

In [10]:
# et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(non_stat_rules), conn)

In [11]:
if len(non_stat_rules) == 1:
    et_risk_df_per_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES]  A with(nolock) inner join FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL B with (nolock) ON A.ORGANIZATION_MASTER_ID = B.ORGANIZATION_MASTER_ID where RULE_NO in ('{}')".format(non_stat_rules[0]),conn)
    
elif(len(non_stat_rules)== 0):
    et_risk_df_per_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES]  A with(nolock) inner join FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL B with (nolock) ON A.ORGANIZATION_MASTER_ID = B.ORGANIZATION_MASTER_ID where RULE_NO in ('')",conn)
    
else:
    non_stat_rules = tuple(non_stat_rules)
    et_risk_df_per_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES]  A with(nolock) inner join FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL B with (nolock) ON A.ORGANIZATION_MASTER_ID = B.ORGANIZATION_MASTER_ID where RULE_NO in" + str (non_stat_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [12]:
et_risk_df_per_rules

Empty DataFrame
Columns: [ORGANIZATION_MASTER_ID, RULE_NO, RULE_VALUE, RISK_SCORE_STATIC, CREATE_DATE]
Index: []

In [14]:
auto_high_df = et_risk_df_per_rules[(et_risk_df_per_rules['RULE_NO'].isin(auto_high_rules)) & (et_risk_df_per_rules['RISK_SCORE_STATIC'] == 10)]

risk_per = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION]",conn)

auto_high_df = auto_high_df.groupby('ORGANIZATION_MASTER_ID')['RULE_NO'].unique().reset_index()

auto_high_df['RISK_RULE_LIST'] = auto_high_df['RULE_NO'].apply(lambda x:"_".join(x))

auto_high_df['RISK_SCORE'] = 10

In [15]:
flaged_per_master_id = auto_high_df['ORGANIZATION_MASTER_ID'].to_list()
flaged_per_master_id

[]

In [16]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [17]:

#Create dataframe using pyodbc

summary_stg_df = pd.read_sql_query("SELECT * FROM FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG WHERE ORGANIZATION_MASTER_ID IN (SELECT ORGANIZATION_Master_Id FROM FINCORE_DB.Fincore_Analytics.risk_organization_incremental)", conn)

In [18]:
summary_stg_df.head()

ORGANIZATION_MASTER_ID   PEER_GROUP_ID         PAN GSTIN  \
0                      78        Others_0  UTYCH8574I  None   
1                      79        Others_0  UTYCH8575I  None   
2                      80  Testing data_0  TTBCK5841J  None   
3                      81        Others_0  UYTCD5432W  None   
4                      82        Others_0  UYTCP5255Y  None   

       UNIQUE_COMPANY_ID   IEC        FULL_NAME DATE_OF_INCORPORATION  \
0  L09631KA2000PTC752842  None     Ram Universe            1952-04-15   
1  L09631KA2000PTC752941  None    Shiv Universe            2001-05-12   
2  L09638KA3000PTC412008  None  Pavansut Energy            2000-05-12   
3  L09631KA2000PTC781003  None  Sugar Cosmetics            2002-05-11   
4  L09631KA2000PTC784102  None   Excellence Ind            2000-01-08   

  CBDT_INCOME_RANGE  TRAN_AMNT_CREDIT  TRAN_AMNT_DEBIT  PEER_GROUP_AMNT  \
0              None               nan              nan          0.00000   
1              None               nan              nan          0.00000   
2              None               nan              nan          0.00000   
3              None               nan   10000000.00000   10000000.00000   
4              None    10000005.00000              nan   10000005.00000   

   PINCODE LATITUDE LONGITUDE OCCUPATION_BUSINESS ACCOUNT_COUNT ADDRESS_COUNT  \
0   560100     None      None                None          None          None   
1   560100     None      None                None          None          None   
2   560100     None      None        Testing data          None          None   
3   560100     None      None                None          None          None   
4   560100     None      None                None          None          None   

   MOBILE_COUNT TELEPHONE_COUNT  EMAIL_COUNT WATCHLIST_COUNT  \
0           nan            None          nan            None   
1           nan            None          nan            None   
2       1.00000            None          nan            None   
3           nan            None          nan            None   
4           nan            None      1.00000            None   

  LEA_REQUESTS_COUNT LNK_REPORT_COUNT STR_REPORT_COUNT STR_AMOUNT_DEBIT  \
0               None             None             None             None   
1               None             None             None             None   
2               None             None             None             None   
3               None             None             None             None   
4               None             None             None             None   

  STR_AMOUNT_CREDIT  CTR_REPORT_COUNT CTR_AMOUNT_DEBIT  CTR_AMOUNT_CREDIT  \
0              None           1.00000             None     40000040.00000   
1              None           1.00000             None     40000040.00000   
2              None               nan             None                nan   
3              None               nan             None                nan   
4              None           1.00000             None     40000020.00000   

   NTR_REPORT_COUNT  NTR_AMOUNT_DEBIT NTR_AMOUNT_CREDIT PTR_REPORT_COUNT  \
0               nan               nan              None             None   
1               nan               nan              None             None   
2           1.00000     2400000.00000              None             None   
3           1.00000    20000000.00000              None             None   
4               nan               nan              None             None   

  PTR_AMOUNT_DEBIT PTR_AMOUNT_CREDIT CBWTR_REPORT_COUNT CBWTR_AMOUNT_DEBIT  \
0             None              None               None               None   
1             None              None               None               None   
2             None              None               None               None   
3             None              None               None               None   
4             None              None               None               None   

  CBWTR_AMOUNT_CREDIT 

In [19]:
temp = pd.merge(auto_high_df,summary_stg_df , on = 'ORGANIZATION_MASTER_ID',how = 'inner')

In [20]:
# temp.rename(columns = {'PEER_GROUP_ID_y':'PEER_GROUP_ID'}, inplace = True)

In [21]:
temp

Empty DataFrame
Columns: [RULE_NO, RISK_RULE_LIST, RISK_SCORE, ORGANIZATION_MASTER_ID, PEER_GROUP_ID, PAN, GSTIN, UNIQUE_COMPANY_ID, IEC, FULL_NAME, DATE_OF_INCORPORATION, CBDT_INCOME_RANGE, TRAN_AMNT_CREDIT, TRAN_AMNT_DEBIT, PEER_GROUP_AMNT, PINCODE, LATITUDE, LONGITUDE, OCCUPATION_BUSINESS, ACCOUNT_COUNT, ADDRESS_COUNT, MOBILE_COUNT, TELEPHONE_COUNT, EMAIL_COUNT, WATCHLIST_COUNT, LEA_REQUESTS_COUNT, LNK_REPORT_COUNT, STR_REPORT_COUNT, STR_AMOUNT_DEBIT, STR_AMOUNT_CREDIT, CTR_REPORT_COUNT, CTR_AMOUNT_DEBIT, CTR_AMOUNT_CREDIT, NTR_REPORT_COUNT, NTR_AMOUNT_DEBIT, NTR_AMOUNT_CREDIT, PTR_REPORT_COUNT, PTR_AMOUNT_DEBIT, PTR_AMOUNT_CREDIT, CBWTR_REPORT_COUNT, CBWTR_AMOUNT_DEBIT, CBWTR_AMOUNT_CREDIT, CCR_REPORT_COUNT, CCR_AMOUNT_DEBIT, CCR_AMOUNT_CREDIT, CREATE_DATE]
Index: []

In [22]:
# Adding the flaged person in auto high rules to risk person table

for ind,row in temp.iterrows():
    try:
        conn.execute('''INSERT INTO [Fincore_Analytics].[RISK_ORGANIZATION] (ORGANIZATION_MASTER_ID, PEER_GROUP_ID, RISK_SCORE)
    values (?,?,?)''',(row['ORGANIZATION_MASTER_ID'],row['PEER_GROUP_ID'],10))
        
    except:
        pass

conn.commit()
conn.close()


In [23]:
et_risk_df_per_rules = et_risk_df_per_rules[~et_risk_df_per_rules['ORGANIZATION_MASTER_ID'].isin(auto_high_df['ORGANIZATION_MASTER_ID'].to_list())]

In [24]:
et_risk_df_per_rules

ORGANIZATION_MASTER_ID     RULE_NO RULE_VALUE RISK_SCORE_STATIC  \
0                       78  O002001007          1              None   
1                       78  O002001024          1              None   
2                       78  O002001027          1              None   
3                       78  O002001028          1              None   
4                       78  O002001030          1              None   
5                       78  O002001033          1              None   
6                       79  O002001007          1              None   
7                       79  O002001024          1              None   
8                       79  O002001027          1              None   
9                       79  O002001028          1              None   
10                      79  O002001030          1              None   
11                      79  O002001033          1              None   
12                      80  O002001007          1              None   
13                      80  O002001008          1              None   
14                      80  O002001024          1              None   
15                      80  O002001027          1              None   
16                      80  O002001030          1              None   
17                      80  O002001033          1              None   
18                      80  O002001034          1              None   
19                      81  O002001001          1              None   
20                      81  O002001002          1              None   
21                      81  O002001004          1              None   
22                      81  O002001007          1              None   
23                      81  O002001024          1              None   
24                      81  O002001027          1              None   
25                      81  O002001030          1              None   
26                      81  O002001033          1              None   
27                      81  O002001034          1              None   
28                      82  O002001001          1              None   
29                      82  O002001002          1              None   
30                      82  O002001004          1              None   
31                      82  O002001007          1              None   
32                      82  O002001009          1              None   
33                      82  O002001024          1              None   
34                      82  O002001027          1              None   
35                      82  O002001028          1              None   
36                      82  O002001030          1              None   
37                      82  O002001033          1              None   
38                      83  O002001001          1              None   
39                      83  O002001002          1              None   
40                      83  O002001004          1              None   
41                      83  O002001007          1              None   
42                      83  O002001009          1              None   
43                      83  O002001024          1              None   
44                      83  O002001027          1              None   
45                      83  O002001028          1              None   
46                      83  O002001030          1              None   
47                      83  O002001033          1              None   

   CREATE_DATE  
0   2023-02-23  
1   2023-02-23  
2   2023-02-23  
3   2023-02-23  
4   2023-02-23  
5   2023-02-23  
6   2023-02-23  
7   2023-02-23  
8   2023-02-23  
9   2023-02-23  
10  2023-02-23  
11  2023-02-23  
12  2023-02-23  
13  2023-02-23  
14  2023-02-23  
15  2023-02-23  
16  2023-02-23  
17  2023-02-23  
18  2023-02-23  
19  2023-02-23  
20  2023-02-23  
21  2023-02-23  
22  2023-02-23  
23  2023-02-23  
24  2023-02-23  
25  2023-02-23  
26  2023-02-23  
27  2023-02-23  
28  2023-02-23  
29  2023-02-23

In [25]:
et_risk_df_per_rules.drop_duplicates(inplace=True)

In [26]:
# et_risk_df_per_rules[et_risk_df_per_rules['PERSON_MASTER_ID'] == 76400]

In [27]:
# t = et_risk_df_per_rules.groupby(['PERSON_MASTER_ID','RULE_NO']).size().reset_index()
# t[t[0]>1]

In [28]:
#changing vertical format to horizontal
pivot_per_df = et_risk_df_per_rules
pivot_per_df = pivot_per_df.pivot(index='ORGANIZATION_MASTER_ID', columns='RULE_NO', values=['RULE_VALUE'])
pivot_per_df.columns = [col[1] for col in pivot_per_df.columns.values]
pivot_per_df = pivot_per_df.reset_index()
cols = pivot_per_df.columns.to_list()
cols.append('CREATE_DATE')
pivot_per_df = pivot_per_df.merge(et_risk_df_per_rules,on = 'ORGANIZATION_MASTER_ID')[cols]



# pivot_org_df = et_risk_df_org_rules
# pivot_org_df = pivot_org_df.pivot(index='ORGANIZATION_MASTER_ID', columns='RULE_NO', values=['RULE_VALUE'])
# pivot_org_df.columns = [col[1] for col in pivot_org_df.columns.values]
# pivot_org_df = pivot_org_df.reset_index()
# cols = pivot_org_df.columns.to_list()
# cols.append('CREATE_DATE')
# pivot_org_df = pivot_org_df.merge(et_risk_df_org_rules,on = 'ORGANIZATION_MASTER_ID')[cols]

In [30]:
data_ind_U = data_ind_U.merge(pivot_per_df,how = 'left',on = 'ORGANIZATION_MASTER_ID')
# et_risk_df_org = et_risk_df_org.merge(pivot_org_df,how='left',on = 'ORGANIZATION_MASTER_ID')
data_ind_U.head()

ORGANIZATION_MASTER_ID  ORGANIZATION_MASTER_ID.1 OCCUPATION_BUSINESS  \
0                      78                        78              Others   
1                      78                        78              Others   
2                      78                        78              Others   
3                      78                        78              Others   
4                      78                        78              Others   

   PEER_GROUP_AMNT PEER_GROUP_ID  PEER_GROUP_AMNT_Scaled O002001001  \
0          0.00000      Others_0                 0.00000        NaN   
1          0.00000      Others_0                 0.00000        NaN   
2          0.00000      Others_0                 0.00000        NaN   
3          0.00000      Others_0                 0.00000        NaN   
4          0.00000      Others_0                 0.00000        NaN   

  O002001002 O002001004 O002001007 O002001008 O002001009 O002001024  \
0        NaN        NaN          1        NaN        NaN          1   
1        NaN        NaN          1        NaN        NaN          1   
2        NaN        NaN          1        NaN        NaN          1   
3        NaN        NaN          1        NaN        NaN          1   
4        NaN        NaN          1        NaN        NaN          1   

  O002001027 O002001028 O002001030 O002001033 O002001034 CREATE_DATE  
0          1          1          1          1        NaN  2023-02-23  
1          1          1          1          1        NaN  2023-02-23  
2          1          1          1          1        NaN  2023-02-23  
3          1          1          1          1        NaN  2023-02-23  
4          1          1          1          1        NaN  2023-02-23

In [31]:
data_ind_U = data_ind_U[~data_ind_U['ORGANIZATION_MASTER_ID'].isin(auto_high_df['ORGANIZATION_MASTER_ID'].to_list())]

In [32]:
data_ind_U.drop(columns = ['ORGANIZATION_MASTER_ID','CREATE_DATE'],inplace = True)

In [33]:
data_ind_U.drop_duplicates(inplace = True)

In [34]:
# for rule in list(set(non_stat_rules) - set(auto_high_rules)):
# #     print(rule)
# #     data_ind_U[rule].fillna(value = 0,inplace = True)
#     data_ind_U[rule] = data_ind_U[rule].astype('int64') 

In [35]:
# data_ind_U.dtypes

In [36]:
# reqd_cols_U = [col for col in data_ind_U.columns if 'nbr' in col.lower()]
reqd_cols_per = list(set(non_stat_rules) - set(auto_high_rules)) #doing this to remove auto high rule from nan_stat_rule_list
#Uncomment to run on org
# reqd_cols_org = ['O002001001','O002001018','O002001017','O002001024', 'O002001025', 'O002001026']



# reqd_cols_U.append('TRAN_AMNT')
# reqd_cols_U

# reqd_cols_U
# reqd_cols = ['Transaction_Amount','Nbr_of_CTR_reports','Nbr_of_STR_Reports','Nbr_of_NTR_reports','WL_Count','Nbr_of_email_ids','Nbr_of_phones']

# Threshold Calculation with Outliers

In [37]:
# def compute_outlier(data, col):
# #     inf = float('inf')
# #     print("Hello",len(data[col]),type(data[col]))
# #     if len(data[col]) >= 30:
#         #if there are more than 30 data points, use z-score approach
    
#     try:    
#         d = data[col].dropna().astype('int64')
#         if len(d) == 0:
# #             return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
#             return None,None,None,None,None,None,None,None,None,None
#         else:
        
            
#             median = np.median(d)
#             # change it to median
#             std = np.std(d)
#     #         print(d,col,"****************")
#             one = round(median+std)
#             two = round(median+2*std)
#             three = round(median+3*std)
#             four = round(median+4*std)
#             five = round(median+5*std)
#             six = round(median+6*std)
#             seven = round(median+7*std)
#             eight = round(median+8*std)
#             nine =round(median+9*std)
#             ten = round(median+10*std)
#             return one,two,three,four,five,six,seven,eight,nine,ten        
#     except KeyError:
# #         return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
#         return None,None,None,None,None,None,None,None,None,None
# #         pass
# #         low = 0
# #         mid = mean
# #         high = np.max(data[col])
# #         low_inc = (mid-low)/5
# #         high_inc = (high-mid)/5
# #         one = low
# #         two = low+low_inc
# #         three = low+2*low_inc
# #         four = low+3*low_inc
# #         five = low+4*low_inc
# #         six = mid
# #         seven = mid+high_inc
# #         eight = mid+2*high_inc
# #         nine = mid+3*high_inc
# #         ten = mid+4*high_inc
    

# Threshold Calculations without Outliers

In [38]:
# def compute_IQR(data):
#     q3 = np.quantile(data, 0.75)
#     q1 = np.quantile(data, 0.25)
#     iqr = ((q3-q1)* 1.5)
#     outliers_left = q1 - iqr
#     outliers_right = q3 + iqr
#     data = list(filter(lambda x: outliers_left <= x <= outliers_right, data))

    
#     return data

In [39]:
# def normal_outliers_removal(d):
#     mean = np.mean(d)
#     std = np.std(d)
#     outliers_left = (mean - (3*std))
#     outliers_right = (mean + (3*std))
#     data = list(filter(lambda x: outliers_left <= x <= outliers_right, d))
#     return d
    
    

In [40]:
# def thresh_cal(d):
#     median = np.median(d)
#     mean = np.mean(d)
#     std = np.std(d)
#     one = round(median+std)
#     two = round(median+2*std)
#     three = round(median+3*std)
#     four = round(median+4*std)
#     five = round(median+5*std)
#     six = round(median+6*std)
#     seven = round(median+7*std)
#     eight = round(median+8*std)
#     nine =round(median+9*std)
#     ten = round(median+10*std)
#     return one,two,three,four,five,six,seven,eight,nine,ten  
    
    

In [41]:
# def not_normal_outliers_removal(d):
#     data = [*set(d)]
#     return compute_IQR(d)

In [42]:


# def compute_outlier(data,col):
#     try:
#         print("data = ",data)
#         d = data[col].dropna().astype('int64')
#         print("d = ",d)
    
#         if (len(d) == 0):
#             return None,None,None,None,None,None,None,None,None,None


#         elif ( len(d) < 5):
#             return thresh_cal(d) 
        
        
        
#         else:
            
#             if ( len(d) <= 150):
#                 stats,p_val = shapiro(d)
#                 if p_val > 0.05:
#                     d = normal_outliers_removal(d)
#                     return thresh_cal(d)
#                 else:
#                     d = not_normal_outliers_removal(d)
#                     return thresh_cal(d)   
                    
#             else:
#                 stats,p_val  = kstest(d, 'norm')
#                 if p_val > 0.05:
#                     d = normal_outliers_removal(d)
#                     return thresh_cal(d)
                    
#                 else:
#                     d = not_normal_outliers_removal(d)
#                     return thresh_cal(d)
    
            
#     except KeyError:
#         return( None,None,None,None,None,None,None,None,None,None) 
    
#     return (None,None,None,None,None,None,None,None,None,None)

In [43]:
# class Thresholds():
#     def __init__(self,df,reqd_cols):
#         self.data = df
#         self.reqd_cols_U = reqd_cols
#     def get_thresholds(self):
#         # Python3 dict to contain threshold calculations-
#         # Python3 lists to contain threshold values for desired columns-       
#         self.class_dict = {}
#         for col in self.reqd_cols_U:
#             self.class_dict[col] = []
        
#         self.class_dict['OCCUPATION_BUSINESS_CLEAN'] = []
#         self.class_dict['Cluster_labels'] = []

#         for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
# #             if occ == 'Others':
#                 for clabel in self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ, 'Cluster_labels'].unique():

#                     for col in self.reqd_cols_U:
#                         print(f"Profession: {occ}, Cluster = {clabel},"
#                         f" col = {col}; outlier"
#                         f" = {compute_outlier(self.data[(self.data['OCCUPATION_BUSINESS_CLEAN'] == occ) & (self.data['Cluster_labels'] == clabel)], col)}"
#                         )
#                         # profession.append(occ)
#                         # profession.append(clabel)
#                         self.class_dict[col].append(compute_outlier(self.data[(self.data['OCCUPATION_BUSINESS_CLEAN'] == occ) & (self.data['Cluster_labels'] == clabel)], col))

#                     self.class_dict['OCCUPATION_BUSINESS_CLEAN'].append(occ)
#                     self.class_dict['Cluster_labels'].append(clabel)
# #                 print(self.class_dict)
# #                 break
# #         self.class_dict['Type_of_profession'] = []
# #         self.class_dict['Cluster_labels'] = []
                        
# #         for occ in self.data['Type_of_profession'].unique():
# #             for clabel in self.data.loc[self.data['Type_of_profession'] == occ, 'Cluster_labels'].unique():
# #                 # print(f"{occ}, {clabel}")
# #                 self.class_dict['Type_of_profession'].append(occ)
# #                 self.class_dict['Cluster_labels'].append(clabel)
        
#         self.threshold_data = pd.DataFrame(self.class_dict)
#         return self.threshold_data
# #         return self.class_dict

In [44]:
# ind_U = Thresholds(data_ind_U,reqd_cols_per)
# ind_data_thresholds_U = ind_U.get_thresholds()

# #Uncomment to run on org
# # org_U = Thresholds(data_org_U,reqd_cols_org)
# # org_data_thresholds_U = org_U.get_thresholds()


In [45]:
# data_ind_U.head()

In [46]:
# ind_data_thresholds_U[(ind_data_thresholds_U['OCCUPATION_BUSINESS_CLEAN'] == 'Business') & (data_ind_U['Cluster_labels'] == 1)]


In [47]:
# #Create dataframe using pyodbc
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()
# # data_wt = cursor.execute("SELECT * FROM [Fincore_Analytics].[RISK_LKP_RULES]").fetchall()

In [48]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [49]:
weight_sql_inc = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_LKP_RULES]", conn)
weight_sql_inc

RULE_ID       RULE_NO                                          RULE_DESC  \
0    3.3.36    T003003036        Multiple deposits made to NGO(s) account(s)   
1    3.3.37    T003003037  Value of deposits made to NGO(s) account(s) ex...   
2    3.3.38    T003003038  Donation from single entity is greater than pa...   
3    3.3.39    T003003039  Multiple donors are contributing more than par...   
4    3.3.43    T003003043  Value of transactions destined towards high ri...   
..      ...           ...                                                ...   
221   2.1.7    O002001007  Number of names/aliases reported across differ...   
222   3.1.8    O003001008                      Organization risk given by RE   
223   3.3.9    T003003009                         Cash at ATM_Frequency Risk   
224   3.3.7  T003003007_D                   Cash at ATM_Value Risk (Deposit)   
225   3.3.8  T003003008_D                   Cash at ATM_Volume Risk(Deposit)   

            PERSON_TYPE                                          RISK_TYPE  \
0                   NTR                                   Transaction Risk   
1                   NTR                                   Transaction Risk   
2                   NTR                                   Transaction Risk   
3                   NTR                                   Transaction Risk   
4                 CBWTR                                   Transaction Risk   
..                  ...                                                ...   
221        Organization  KYC profile/ Inherent risk based High Risk Ind...   
222        Organization                             KYC profile based Risk   
223       TC1_Frequency                                   Transaction Risk   
224   TC1_Value_Deposit                                   Transaction Risk   
225  TC1_Volume_Deposit                                   Transaction Risk   

     IS_ACTIVE_FLAG CREATE_DATE UPDATE_DATE SOURCE_DATA_EXISTING_UNDRSTD  \
0                 1  2022-02-03        None                         None   
1                 1  2022-02-03        None                         None   
2                 0  2022-02-03        None                         None   
3                 0  2022-02-03        None                         None   
4                 1  2022-02-03        None                         None   
..              ...         ...         ...                          ...   
221               1  2022-02-03        None                         None   
222               1  2022-02-03        None                         None   
223               1  2022-02-03        None                         None   
224               1  2022-02-03        None                         None   
225               1  2022-02-03        None                         None   

        DEVELOPMENT_STATUS CODE_REVIEW_STATUS  \
0    Development Completed               None   
1    Development Completed               None   
2                  On Hold               None   
3                  On Hold               None   
4    Development Completed               None   
..                     ...                ...   
221      Testing Completed               None   
222      Testing Completed               None   
223       Sent for Testing               None   
224       Sent for Testing               None   
225       Sent for Testing               None   

                                            DEPENDENCY  WEIGHTAGE  \
0                                                 None          5   
1                                                 None          5   
2                                                 None          5   
3                                                 None          5   
4                                                 None          8   
..                                                 ...        ...   
221                                               None          6   
222  FUNCTIONAL CLARITY|Need Inputs from Functional.

In [50]:
# weight_table_ind_U_inc = ind_data_thresholds_U.copy()
# #uncomment to run for org
# # weight_table_org_U = org_data_thresholds_U.copy()

# # weight_table_ind = ind_data_thresholds.copy()
# # weight_table_org = org_data_thresholds.copy()

In [51]:
# for col in ind_data_thresholds_U.columns:
#     if (col == 'OCCUPATION_BUSINESS_CLEAN') or (col == 'Cluster_labels') or (col == 'TRAN_AMNT'):
#         pass
#     else:
#         weight_table_ind_U[col] = weight_sql[weight_sql['RULE_NO'] == col]['WEIGHTAGE'].values[0]

# #uncomment to run for org
# # for col in org_data_thresholds_U.columns:
# #     if (col == 'OCCUPATION_BUSINESS_CLEAN') or (col == 'Cluster_labels') or (col == 'TRAN_AMNT'):
# #         pass
# #     else:
# #         weight_table_org_U[col] = weight_sql[weight_sql['RULE_NO'] == col]['WEIGHTAGE'].values[0]



In [52]:
# pyodbc.drivers()
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# rule_df = pd.read_sql_query("SELECT * FROM [HDM_FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] where IS_ACTIVE_FLAG = 1 and RISK_SCORE_STATIC_FLAG = 0 and RISK_TYPE <> 'Auto High Rules' and PERSON_TYPE = 'Individual'", conn)
# reqd_cols_per = rule_df['RULE_NO'].to_list()
# len(reqd_cols_per)

In [53]:
# col_with_data = data_ind_U.columns.to_list()

In [54]:
# col_with_data = [col for col in col_with_data if 'P0' in col]

In [55]:
# col_with_data

In [56]:
# reqd_cols_per

In [57]:
# col_without_data = set(reqd_cols_per) - set(col_with_data)

In [58]:
# col_without_data

In [59]:
# ind_data_thresholds_U.to_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\threshold_individual_U_inc.csv",index=False)
#Uncomment to run on org
# org_data_thresholds_U.to_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\threshold_organization_U.csv",index=False)

data_ind_U.to_csv(r"/data/Synthetic_data_Clustered_individual_U_inc.csv",index=False)

# weight_table_ind_U.to_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\weight_individual_U_inc.csv",index=False)
# weight_table_org_U.to_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\weight_organization_U.csv",index=False)


In [60]:
# data_ind_U.to_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\Synthetic_data_Clustered_individual_U.csv",index=False)
# data_ind_U.head(5)

In [61]:
print("Threshold_Computation_V2_Incremental_data Completed")

Threshold_Computation_V2_Incremental_data Completed
